In [1]:
import os
import pandas as pd
import json
import requests
from meteostat import Daily, Stations
from datetime import datetime

In [ ]:
def population_data(df):
    '''
    Fügt jeder Stadt Einwohner hinzu
    Merge über Kalenderjahr
    '''
    df = df.copy()
    
    df_population = pd.read_csv(
                    './data/population.csv',
                    sep=',', 
                    header=0,
                    engine='python',  # Hilft oft bei Parsing-Problemen
                    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
                    encoding='utf-8')  # Falls Sonderzeichen vorhanden sind


    df_population = df_population[['City', 'Value', 'Year']]
    df_population.rename(columns={'Value': 'Population'}, inplace=True)
    df_population.dropna(inplace=True)
    df_population['Population'] = df_population['Population'].astype(int)

    # Sicherstellen, dass die Spaltennamen übereinstimmen
    df.columns = df.columns.str.capitalize()
    df['Year'] = df['Year'].astype(int)
    df_population['Year'] = df_population['Year'].astype(int)

    # Sortieren für merge_asof (wichtig!)
    df = df.sort_values(by=['City', 'Year'])
    df_population = df_population.sort_values(by=['City', 'Year'])

    # Merge mit nächstgelegenem Jahr
    df = pd.merge_asof(df, df_population, on='Year', by='City', direction='nearest')

    # Datei speichern
    output_path = './data/population_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    return df